# 앙상블 모델

In [ ]:
# 연산 처리를 위한 패키지
import numpy as np
import pandas as pd

# 데이터 분석을 위한 패키지
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_validate

# 시각화를 위한 패키지
from matplotlib import pyplot as plt
import seaborn as sns

# 그래프를 실제로 그리기 위한 설정
%matplotlib inline

# 경고 메세지 무시
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('./data/classification/df_titanic.csv')
df.head()

In [ ]:
grid_learn = [0.0001, 0.001, 0.01, 0.03, 0.05, 0.1, 0.2, 0.25, 0.5, 1.0]
grid_n_estimator = [10, 50, 100, 300, 500]
grid_ratio = [0.1, 0.25, 0.5, 0.75, 1.0]
grid_max_features = [0.3, 0.5, 0.7, 1.0]
grid_max_depth = [1, 2, 4, 8]
grid_min_samples_leaf = [1, 2, 3, 10, 100, 1500]
grid_min_samples_split = [2, 4, 8, 16, 24, 30]
grid_seed = [0]

## 배깅(Bagging)

In [ ]:
feature_columns = (df.columns.difference(['Survived']))

X = df[feature_columns]
y = df['Survived']

# Bagging - 파라미터 튜닝 없이
scores = cross_val_score(BaggingClassifier(random_state=0), X, y, scoring='accuracy', cv=5)
print("Bagging 평균 정확도:", np.round(np.mean(scores),4))

- cross_val_score()
    - 해당 함수 내부에서 학습(fit), 예측(predict), 평가(evaluation) 그리고 교차 검증을 수행해줍니다.

In [ ]:
scores = cross_validate(BaggingClassifier(random_state=0), X, y, scoring=['accuracy', 'roc_auc'], return_train_score=True, cv=5)
np.transpose(pd.DataFrame(scores))

In [ ]:
print("Bagging train 평균 정확도:", np.round(np.mean(scores['train_accuracy']),4))
print("Bagging test 평균 정확도:", np.round(np.mean(scores['test_accuracy']),4))

- cross_validate() 함수
    - 각 폴드에서 훈련과 테스트에 걸린 시간을 확인 할 수 있습니다.
    - 훈련(train) 점수, 테스트(test) 점수를 함께 볼 수 있습니다. 

### 하이퍼 파라미터
- 디폴트(default) 설정
```python
BaggingClassifier(
    base_estimator=None,
    n_estimators=10,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=True,
    bootstrap_features=False,
    oob_score=False,
    warm_start=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
)
```
- base_estimator: 예측할 모델, 디폴트는 decision tree
- n_estimators: 모형의 갯수
- max_samples: 각각의 예측기가 X에서 추출 할 샘플의 수 
- max_features: 각각의 예측기가 X에서 가져갈 컬럼의 수
- bootstrap: 데이터 중복 여부, 디폴트는 True

### 배깅 - GridSearchCV 를 활용한 최적의 파라미터 찾기

In [ ]:
bc_param = {
    'n_estimators': grid_n_estimator,  # default=10
    'max_samples': grid_ratio,  # default=1.0
    'random_state': grid_seed,
    'max_features': grid_max_features,
}

# n_jobs = -1 학습시 모든 CPU 코어 사용
grid_bc = GridSearchCV(BaggingClassifier(), param_grid=bc_param, 
                       return_train_score=True, n_jobs = -1, 
                       cv=5, scoring='accuracy')
grid_bc.fit(X, y)

In [ ]:
BaggingClassifier?

In [ ]:
df_grid_bc = pd.DataFrame(grid_bc.cv_results_)
df_grid_bc[['param_n_estimators', 'param_max_samples','params', 
            'mean_train_score', 'mean_test_score', 
            'rank_test_score']].sort_values(['rank_test_score']).head(5)

In [ ]:
print("최고 Score: ", str(np.round(grid_bc.best_score_ ,4)))
print("최적의 Parameters: ",str(grid_bc.best_params_))
print("최적의 Estimators: ",str(grid_bc.best_estimator_))

In [ ]:
# 최적의 설정값 입력
best_grid_bc = grid_bc.best_estimator_
best_grid_bc.fit(X, y)

pred_bc = best_grid_bc.predict(X)
accuracy_bc = accuracy_score(y, pred_bc)
print(np.round(accuracy_bc,4))

## 랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random forest - 파라미터 튜닝 없이
scores = cross_validate(RandomForestClassifier(random_state=0), X, y, 
                        scoring=['accuracy', 'roc_auc'], 
                        return_train_score=True, cv=5)
np.transpose(pd.DataFrame(scores))

In [ ]:
print("Random forest train 평균 정확도:", np.round(np.mean(scores['train_accuracy']),4))
print("Random forest test 평균 정확도:", np.round(np.mean(scores['test_accuracy']),4))

### 하이퍼 파라미터

In [ ]:
RandomForestClassifier?

### 랜덤 포레스트 - GridSearchCV 를 활용한 최적의 파라미터 찾기

In [ ]:
rf_param = {
    'n_estimators': grid_n_estimator,
    'random_state': grid_seed,
    'max_depth': grid_max_depth,
    'min_samples_leaf': grid_min_samples_leaf,
    'min_samples_split': grid_min_samples_split
}

# n_jobs = -1 학습시 모든 CPU 코어 사용
grid_rf = GridSearchCV(RandomForestClassifier(random_state=0), param_grid=rf_param, 
                       return_train_score=True,n_jobs=-1, cv=5, scoring='accuracy')
grid_rf.fit(X, y)

In [ ]:
df_grid_rf = pd.DataFrame(grid_rf.cv_results_)
df_grid_rf[['params','mean_train_score','mean_test_score',
            'rank_test_score']].sort_values(['rank_test_score']).head(5)

In [ ]:
print("최고 Score: ", str(np.round(grid_rf.best_score_ ,4)))
print("최적의 Parameters: ",str(grid_rf.best_params_))
print("최적의 Estimators: ",str(grid_rf.best_estimator_))

In [ ]:
# 최적의 설정값 입력
best_grid_rf = grid_rf.best_estimator_
best_grid_rf.fit(X, y)

pred_rf = best_grid_rf.predict(X)
accuracy_rf = accuracy_score(y, pred_rf)
print(np.round(accuracy_rf,4))

In [ ]:
# 가장 중요한 변수
feature_names = list(X.columns)

plt.title("RandomForest Feature importances for Survival")
sns.barplot(x=best_grid_rf.feature_importances_, y=feature_names)

## Boosting

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# AdaBoost - 파라미터 튜닝 없이
scores = cross_validate(AdaBoostClassifier(random_state=0), X, y, scoring=['accuracy', 
                                                                           'roc_auc'], return_train_score=True, cv=5)
np.transpose(pd.DataFrame(scores))

In [ ]:
print("Adaboost train 평균 정확도:", np.round(np.mean(scores['train_accuracy']),4))
print("Adaboost test 평균 정확도:", np.round(np.mean(scores['test_accuracy']),4))

#### 하이퍼 파라미터
- 디폴트(default) 설정

```python
AdaBoostClassifier(
    base_estimator=None,
    n_estimators=50,
    learning_rate=1.0,
    algorithm='SAMME.R',
    random_state=None,
)
```
- base_estimator: 예측할 모델, 디폴트는 DecisionTreeClassifier(max_depth=1)
- n_estimators: 모형(week learner)의 갯수, 순차적으로 오류를 보정해서 수가 많으면 성능이 일정 수준까지 높아 질 수 있으나 수행 시간이 오래 걸린다는 단점이 있음. 디폴트는 50
- learning_rate: 학습률, 0~1 사이의 값을 지정. 너무 작은 값인 경우 최소점을 찾아 예측 성능이 높지만 학습에 오래 걸리고 너무 큰 값인 경우 최소점을 찾지 못해 예측 성능이 떨어질 확률이 높음. 그래서 n_estimators와 상호 호환 필요. 디폴트는 1.0

In [ ]:
AdaBoostClassifier?

#### AdaBoost - GridSearchCV 를 활용한 최적의 파라미터 찾기

In [ ]:
ada_param = {
            'n_estimators': grid_n_estimator, 
            'learning_rate': grid_learn, 
            'random_state': grid_seed
}

# n_jobs = -1 학습시 모든 CPU 코어 사용
grid_ada = GridSearchCV(AdaBoostClassifier(), param_grid=ada_param, 
                        return_train_score=True, n_jobs = -1, 
                        cv=5, scoring='accuracy')
grid_ada.fit(X, y)

In [ ]:
df_grid_ada = pd.DataFrame(grid_ada.cv_results_)
df_grid_ada[['param_learning_rate', 'param_n_estimators','params',
             'mean_train_score','mean_test_score',
             'rank_test_score']].sort_values(['rank_test_score']).head(5)

In [ ]:
print("최고 Score: ", str(np.round(grid_ada.best_score_ ,4)))
print("최적의 Parameters: ",str(grid_ada.best_params_))
print("최적의 Estimators: ",str(grid_ada.best_estimator_))

In [ ]:
# 최적의 설정값 입력
best_grid_ada = grid_ada.best_estimator_
best_grid_ada.fit(X, y)

pred_ada = best_grid_ada.predict(X)
accuracy_ada = accuracy_score(y, pred_ada)
print(np.round(accuracy_ada,4))

In [ ]:
# 가장 중요한 변수
feature_names = list(X.columns)

plt.title("AdaBoosting Feature importances for Survival")
sns.barplot(x=best_grid_ada.feature_importances_, y=feature_names)

### GradientBoost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# GradientBoost - 파라미터 튜닝 없이
scores = cross_validate(GradientBoostingClassifier(random_state=0), X, y, 
                        scoring=['accuracy', 'roc_auc'], 
                        return_train_score=True, cv=5)
np.transpose(pd.DataFrame(scores))

In [ ]:
print("GradientBoost train 평균 정확도:", np.round(np.mean(scores['train_accuracy']),4))
print("GradientBoost test 평균 정확도:", np.round(np.mean(scores['test_accuracy']),4))

In [ ]:
GradientBoostingClassifier?

#### 하이퍼 파라미터
- 디폴트(default) 설정

```python
GradientBoostingClassifier(
    loss='deviance',
    learning_rate=0.1,
    n_estimators=100,
    subsample=1.0,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    init=None,
    random_state=None,
    max_features=None,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    presort='auto',
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
)
```
- loss: 손실 함수(loss function), 디폴트는 'deviance'
- base_estimator: 예측할 모델, 디폴트는 DecisionTreeClassifier(max_depth=1)
- n_estimators: 모형(week learner)의 갯수, 순차적으로 오류를 보정해서 수가 많으면 성능이 일정 수준까지 높아 질 수 있으나 수행 시간이 오래 걸린다는 단점이 있음. 디폴트는 100
- learning_rate: 학습률, 0~1 사이의 값을 지정. 너무 작은 값인 경우 최소점을 찾아 예측 성능이 높지만 학습에 오래 걸리고 너무 큰 값인 경우 최소점을 찾지 못해 예측 성능이 떨어질 확률이 높음. 그래서 n_estimators와 상호 호환 필요. 디폴트는 0.1
- min_samples_leaf: 말단 리프 노드의 최소한의 샘플 데이터 수, 디폴트 1
- max_depth: 트리의 최대 깊이, 디폴트 3
- subsample: n_estimator 모형(week learner)이 학습에 사용하는 데이터의 샘플링 비율, 디폴트 1.0

#### GradientBoost - GridSearchCV 를 활용한 최적의 파라미터 찾기

In [ ]:
gd_param = {
    'n_estimators': grid_n_estimator,
    'learning_rate': grid_learn,
    'random_state': grid_seed,
    'max_depth': grid_max_depth,  # default=3
    'min_samples_leaf': grid_min_samples_leaf,
}

# n_jobs = -1 학습시 모든 CPU 코어 사용
grid_gd = GridSearchCV(GradientBoostingClassifier(), param_grid=gd_param, return_train_score=True, 
                       n_jobs=-1, cv=5, scoring='accuracy')
grid_gd.fit(X, y)

- GradientBoost의 경우 학습을 최적화하기 위한 파라미터가 GridSearchCV 학습이 오래 걸립니다. 
- 파라미터를 조금 조정해서 학습하는 것도 방법

In [ ]:
df_grid_gd = pd.DataFrame(grid_gd.cv_results_)
df_grid_gd[['param_learning_rate', 'param_n_estimators','params',
            'mean_train_score','mean_test_score',
            'rank_test_score']].sort_values(['rank_test_score']).head(5)

In [ ]:
print("최고 Score: ", str(np.round(grid_gd.best_score_ ,4)))
print("최적의 Parameters: ",str(grid_gd.best_params_))
print("최적의 Estimators: ",str(grid_gd.best_estimator_))

In [ ]:
# 최적의 설정값 입력
best_grid_gd = grid_gd.best_estimator_
best_grid_gd.fit(X, y)

pred_gd = best_grid_gd.predict(X)
accuracy_gd = accuracy_score(y, pred_gd)
print(np.round(accuracy_gd,4))

In [ ]:
# 가장 중요한 변수
feature_names = list(X.columns)

plt.title("GradientBoosting Feature importances for Survival")
sns.barplot(x=best_grid_gd.feature_importances_, y=feature_names)

# 분류 모형의 평가와 비교

## Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(y, pred_rf, labels=[1,0])

- True Positive(TP)  | False Negative(FN)
- False Positive(FP) | True Negative(TN)
- TP: 생존을 생존이라고 정확히 예측
- TN: 생존이 아닌것을 생존이 아니라고 정확히 예측
- FP: 생존을 생존이 아니라고 잘못 예측
- FN: 생존이 아닌 것을 생존이라고 잘못 예측


In [ ]:
Confusion_Matrix = pd.crosstab(y, pred_rf, rownames=['Observed'], colnames=['Predicted'])
Confusion_Matrix = Confusion_Matrix[[1,0]]
Confusion_Matrix = Confusion_Matrix.reindex(index =[1,0])
Confusion_Matrix

In [ ]:
sns.heatmap(Confusion_Matrix, annot=True, cmap = 'YlGnBu', fmt = 'd', annot_kws={"size":20})
plt.show()

## Classification_report(정밀도, 재현율, F1 스코어)

- Total = TP + FP + FN + TN 
- Accuracy(정확도): (TP + TN) / Total
- Recall(재현율): TP / (TP + FN)
- Precision(정밀도): TP / (TP + FP)
- F1-score: 2 * Precision * Recall / (Precision + Recall)


In [ ]:
total = 330 + 47 + 85 + 581
accuracy = (330 + 581) / total
recall = 330 / (330 + 85)
precision = 330 / (330 + 47)
f1_score = 2 * precision * recall / (precision + recall)

print("accuracy =", np.round((accuracy),2))
print("recall =", np.round((recall),2))
print("precision =", np.round((precision),2))
print("f1_score =", np.round((f1_score),2))

In [ ]:
print(classification_report(y, pred_rf, target_names=['Survived 0','Survived 1']))

## ROC/AUC Curve
- https://www.scikit-yb.org/en/latest/api/classifier/rocauc.html

In [ ]:
from yellowbrick.classifier import ROCAUC

visualizer = ROCAUC(best_grid_rf, classes=[0, 1])
visualizer.fit(X, y)   
visualizer.score(X, y) 
visualizer.show()    

In [ ]:
visualizer = ROCAUC(best_grid_gd, classes=[0, 1])
visualizer.fit(X, y)
visualizer.score(X, y)
visualizer.show()

In [ ]:
visualizer = ROCAUC(best_grid_ada, classes=[0, 1])

visualizer.fit(X, y)
visualizer.score(X, y)
visualizer.show()